Goal is to develop machine learning code that takes input photon data from an array of my hodoscope and outputs where the beam impinged on the scintillating crystal.

Google Colab Installation of mlinphysics

In [18]:
COLAB_FOLDER = 'MLP' # change as needed
GITHUB_USER  = 'hbprosper'
GITHUB_REPO  = 'mlinphysics'
GITHUB_FOLDERS = ['mlinphysics']
#------------------------------------------------------
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    print('\nGoogle Drive mounted\n')
    IN_COLAB = True
except:
    print('\nRunning locally\n')
    IN_COLAB = False

if IN_COLAB:
    MYDRIVE     = '/content/gdrive/MyDrive'
    GITHUB_BASE = 'https://raw.githubusercontent.com'
    MAIN        = 'refs/heads/main'
    GITHUB_PATH = f'{MYDRIVE}/{COLAB_FOLDER}'
    #------------------------------------------------------
    %cd {GITHUB_PATH}
    %rm -f {GITHUB_PATH}/clone2colab.ipynb
    !wget -q {GITHUB_BASE}/{GITHUB_USER}/{GITHUB_REPO}/{MAIN}/clone2colab.ipynb
    %run {GITHUB_PATH}/clone2colab.ipynb

    %pip install torch_geometric


Running locally



Importing the required modules

In [19]:
# standard system modules
import os, sys

# standard module for tabular data
import pandas as pd

# standard module for array manipulation
import numpy as np

# standard module for high-quality plots
import matplotlib as mp
import matplotlib.pyplot as plt

# standard research-level machine learning toolkit from Meta (FKA: FaceBook)
import torch
import torch.nn as nn
import torch.utils.data as dt

# module to access data in Hieracrchical Data Format (HDF or H5 format)
import h5py

# module to plot pixelized images
import imageio.v3 as im

# module to reimport Python modules
import importlib

# module for saving/loading serialized Python objects
import joblib

# module for shell utilities
import shutil

# ML in physics module
import mlinphysics.nn as mlp
import mlinphysics.utils.data as dat
import mlinphysics.utils.monitor as mon





Computational Device

In [20]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'\n\tAvailable device: {str(DEVICE):4s}\n')


	Available device: cpu 



$\underline{\text{Loading the data sets}}$

In [21]:
produced_photons_array = np.loadtxt('scan_CsI_Se82_master_produced_photon.dat')
absorbed_photons_array = np.loadtxt('scan_CsI_Se82_master_absorbed_photon.dat')
observed_photon_array_array = np.loadtxt('scan_CsI_Se82_master_observed_photon_array.dat')
observed_photon_total_array = np.loadtxt('scan_CsI_Se82_master_observed_photon_total.dat')
beam_position_array = np.loadtxt('scan_CsI_Se82_master_beam_position.dat')

#Normalizing each PMT of the array to the total number of photons seen by the PMTs
normalized_PMT_array = np.zeros((1296,16))

for i in range (1296):
    for m in range (16):
        normalized_PMT_array[i][m] = observed_photon_array_array[i][m] / observed_photon_total_array[i]

# print (normalized_PMT_array.shape)

print(beam_position_array)


[[-3.5 -3.5 -8.   0.   0.   1. ]
 [-3.5 -3.3 -8.   0.   0.   1. ]
 [-3.5 -3.1 -8.   0.   0.   1. ]
 ...
 [ 3.5  3.1 -8.   0.   0.   1. ]
 [ 3.5  3.3 -8.   0.   0.   1. ]
 [ 3.5  3.5 -8.   0.   0.   1. ]]


$\underline{\text{Separating the data sets into training, validating, and testing}}$

In [22]:
# generates a random sequence of integers from 0-1295
randomized_indices = np.random.permutation(1296)

training_indices = randomized_indices[0:648]
validation_indices = randomized_indices[648:972]
evaluating_model_indicies = randomized_indices[972:1296]

# training, validation, and evaluation photon datasets
training_photon_arrays = np.zeros((648,16))
validation_photon_arrays = np.zeros((324,16))
evaluation_photon_arrays = np.zeros((324,16))

# training, validation, and evaluation position datasets
training_position_arrays = np.zeros((648,2))
validation_position_arrays = np.zeros((324,2))
evaluation_position_arrays = np.zeros((324,2))

for i in range (648):
    # filling the training photon datasets
    training_photon_arrays[i][:] = normalized_PMT_array[training_indices[i]][:]

    # filling the training position datasets
    training_position_arrays[i][:] = beam_position_array[training_indices[i], 0:2]
    

for i in range (324):
    # filling the validation and evaluation (testing) photon datasets
    validation_photon_arrays[i][:] = normalized_PMT_array[validation_indices[i]][:]
    evaluation_photon_arrays[i][:] = normalized_PMT_array[evaluating_model_indicies[i]][:]

    # filling the validation and evaluation (testing) position datasets
    validation_position_arrays[i][:] = beam_position_array[validation_indices[i]][0:2]
    evaluation_position_arrays[i][:] = beam_position_array[evaluating_model_indicies[i]][0:2]

#reshaping the normalized data so that we have 432 images of 4x4 pixels
training_photons = training_photon_arrays.reshape(648, 4, 4)
validation_photons = validation_photon_arrays.reshape(324, 4, 4)
evaluating_photons = evaluation_photon_arrays.reshape(324, 4, 4)

print('training idices: ',training_indices[0])

print('training position', training_position_arrays[0][:])

print ('training photons', training_photon_arrays[0][:])
###############################################

# REMEMBER THAT IF YOU WANT TO COMPARE THE INDEX THAT YOU GET WITH THE DATA SET,
# YOU MUST ADD +1 TO THE INDEX VALUE. THE POSSIBLE NUMBERS RANGE FROM 0 TO 1296-1 (1295)
# WHILE THE DATA SET STARTS FROM 1

###############################################


training idices:  743
training position [0.5 1.1]
training photons [0.06548553 0.06799716 0.06473239 0.06271287 0.06586108 0.0681805
 0.06479305 0.06316339 0.06308296 0.06423892 0.06205037 0.05987749
 0.05740199 0.05946105 0.05641711 0.05454413]


$\underline{\text{Preparing the datasets so that they are in the shape of (N, C, H, W)}}$

$\newline$
N - number of images
$\newline$
C - number of input channels 
$\newline$
H - height of our image in pixels
$\newline$
W - width of our image in pixels 

In [23]:
training_photons = training_photons.reshape(648, 1, 4, 4)
validation_photons = validation_photons.reshape(324, 1, 4, 4)
evaluating_photons = evaluating_photons.reshape(324, 1, 4, 4)

$\underline{\text{Configuration of the model}}$

According to this stackexchange conversation

https://stackoverflow.com/questions/4752626/epoch-vs-iteration-when-training-neural-networks

one epoch = one forward pass and one backward pass of all the training examples

batch size = the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you'll need

number of iterations = number of passes, each pass using [batch size] number of examples. 
    To be clear, one pass = one forward pass + one backward pass (forward and backward passes are not counted as two different passes)

For example: if you have 1000 training examples, and your batch size is 500, then it will take 2 iterations to complete 1 epoch.


In [24]:

model_name = 'impinging_position'

# choose whether to create or load a configuration file
load_existing_config = False

if load_existing_config:
    config = mlp.Config(f'{model_name}.yaml')
else:
    # create new config
    config = mlp.Config(model_name)

    n_images = 1296
    batch_size = 18
    n_iters_per_epoch = 36 # number of iterations per epoch
    train_size = n_iters_per_epoch * batch_size
    test_size = 324

    val_size = n_images - train_size - test_size

    config('batch_size', batch_size)
    config('train_size', train_size)
    config('test_size', test_size)
    config('val_size', val_size)

    config('monitor_step', 27) # set monitor training every n
    config('delete', True) # delete losses file before training, if True
    config('frac', 0.015) # save model if average loss decreases by more than frac percent

    config('n_epochs', 200)
    config('n_iters_per_epoch', n_iters_per_epoch)
    config('n_iterations', config('n_epochs') * config('n_iters_per_epoch'))

    config('n_steps', 4) # number of training steps
    config('n_iters_per_step', config('n_iterations') // config('n_steps'))
    
    config('base_lr', 1.e-3) # initial learning rate
    config('gamma', 0.8) # learning rate scale factor

    print(f'\nSave configuration to file {config.cfg_filename}\n')
    
    config.save()

print(config)



Save configuration to file runs/2025-11-20_1431/impinging_position_config.yaml

name: impinging_position
file:
  losses: runs/2025-11-20_1431/impinging_position_losses.csv
  params: runs/2025-11-20_1431/impinging_position_params.pth
  init_params: runs/2025-11-20_1431/impinging_position_init_params.pth
  plots: runs/2025-11-20_1431/impinging_position_plots.png
batch_size: 18
train_size: 648
test_size: 324
val_size: 324
monitor_step: 27
delete: true
frac: 0.015
n_epochs: 200
n_iters_per_epoch: 36
n_iterations: 7200
n_steps: 4
n_iters_per_step: 1800
base_lr: 0.001
gamma: 0.8



$\underline{\text{Getting our data turned into the proper data set so that it can be used with the data loaders}}$

In [25]:
importlib.reload(dat)

train_size = config('train_size')
val_size = config('val_size')
test_size = config('test_size')

# training dataset (this defines the empirical risk to be minimized)
print('training data')
train_data = dat.Dataset(training_photons, start = 0, end = train_size, targets = training_position_arrays)

# a random subset of the training data to check for overtraining
# by comparing with the empirical risk from the validation set

print('training data for validation')
train_data_val = dat.Dataset(training_photons, start = 0, end = train_size, targets = training_position_arrays, random_sample_size = val_size)

# validation dataset (for monitoring training)
print('validation data')
val_data = dat.Dataset(validation_photons, start = 0, end = val_size, targets = validation_position_arrays)

# test dataset
print('test data')
test_data = dat.Dataset(evaluating_photons, start = 0, end = test_size, targets = evaluation_position_arrays)

training data
Dataset
  shape of x: torch.Size([648, 1, 4, 4])
  shape of y: torch.Size([648, 2])

training data for validation
Dataset
  shape of x: torch.Size([324, 1, 4, 4])
  shape of y: torch.Size([324, 2])

validation data
Dataset
  shape of x: torch.Size([324, 1, 4, 4])
  shape of y: torch.Size([324, 2])

test data
Dataset
  shape of x: torch.Size([324, 1, 4, 4])
  shape of y: torch.Size([324, 2])



$\underline{\text{Now we include the data loaders}}$

In [27]:
print('train data loader')
train_laoder = dt.DataLoader(train_data, batch_size = config('batch_size'), shuffle = True)

print('train data loader for validation')
train_loader_val = dt.DataLoader(train_data_val, batch_size = len(train_data_val))

print('validation data loader')
val_loader = dt.DataLoader(val_data, batch_size = len(val_data))

print('test data loader')
test_loader = dt.DataLoader(test_data, batch_size = len(test_data))

train data loader
train data loader for validation
validation data loader
test data loader


$\underline{\textbf{Now we build our model}}$

In [28]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, padding = 1, dropout = 0.08):
        super().__init__()

        kernel_size = 2 * padding + 1

        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, 
                               kernel_size = kernel_size, stride = 1, padding = padding, padding_mode = 'replicate')
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x
    
class BeamPositionIdentifier(mlp.Model):
    def __init__(self, image_size = 4, channels = (1, 8, 16), padding = 1, n_outputs = 2):
        super().__init__()

        # number of convolutional layers
        self.nlayers = len(channels) - 1

        #building convolution layers
        self.convs = nn.ModuleList(
            [ConvBlock(channels[i], channels[i+1], padding)
             for i in range(self.nlayers)])
        
        # computing size after convolutions
        final_image_size = image_size
        ninputs = channels[-1] * (final_image_size**2)

        # Regression (Rather than using maxpool as we want to predict where the beam impinges upon the scintillating crystal (x,y) )
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(ninputs, 64),
            nn.ReLU(),
            nn.Linear(64, n_outputs)
        )


    def forward(self, x):
        for conv in self.convs:
            x = conv(x)
        return self.regressor(x)

$\underline{\text{Instantiate model}}$

In [29]:
importlib.reload(mlp)

model = BeamPositionIdentifier().to(DEVICE)
print(model)
print()

print('number of parameters: ', mlp.number_of_parameters(model))
print()

BeamPositionIdentifier(
  (convs): ModuleList(
    (0): ConvBlock(
      (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
      (relu): ReLU()
      (dropout): Dropout(p=0.08, inplace=False)
    )
    (1): ConvBlock(
      (conv1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=replicate)
      (relu): ReLU()
      (dropout): Dropout(p=0.08, inplace=False)
    )
  )
  (regressor): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=256, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=2, bias=True)
  )
)

number of parameters:  17826



$\underline{\text{Mean Squared Error Loss function}}$

$\newline$
Using the MSE rather than the Average cross entropy loss since we are using regression rather than classification

In [ ]:
class MeanSquaredErrorLoss():
    def __init__(self):
        pass
    def __call__(self, outputs, targets):

        loss = nn.MSELoss()

        result = loss(outputs, targets)

        return result

$\underline{\text{Instantiate training objects}}$

$\newline$
1. optimizer
$\newline$
2. scheduler
$\newline$
3. objective

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = config('base_lr'))

scheduler = mlp.get_stepler_scheduler(optimizer, config)

objective = mlp.Objective(model, MeanSquaredErrorLoss())

In [ ]:
def accuracy(outputs, targets):
    number_correct = float (np.mean(outputs == targets.data.cpu().numpy()))
    return number_correct

$\underline{\text{Defining the Trainer}}$

In [ ]:
def train(objective, optimizer, scheduler, train_loader, train_small_loader, val_loader, config):
    
    # get configuration info
    lossfile = config('file/losses')
    paramsfile = config('file/params')
    step = config('monitor_step')
    delete = config('delete')
    frac = config('frac')
    nepochs = config('n_epochs')
    niters = config('n_iterations')

    # instantiate objects taht saves MSE losses to a csv file for realtime monitoring

    losswriter = mon.LossWriter(niters, lossfile, step = step, frac = frac, model = objective.model, paramsfile = paramsfile)

    # instantiate learning rate step scheduler
    lrscheduler = mlp.LRStepScheduler(optimizer, scheduler)

    # ---------------------------------------
    # training loop
    # ---------------------------------------
    ii = -1

    for epoch in range (nepochs):
        for x, y in train_loader:
            ii += 1

            # set mode to training so that training-specific
            # operations such as dropout, etc., are enabled.
            objective.train()

            # clear all gradients
            optimizer.zero_grad()

            # compute empirical risk
            R = objective(x,y)

            # compute gradients
            R.backward()

            # take one step downhill in the empirical risk landscape
            optimizer.step()

            # check wheter to update learning rate
            lrscheduler.step()

            # I'm alive printout
            if (ii % step == 0) or (ii == niters -1):
                # compute average losses on training and validation data
                t_loss = mlp.compute_avg_loss(objective, train_small_loader)
                v_loss = mlp.compute_avg_loss(objective, val_loader)
                # return current learning rate
                lr = lrscheduler.lr()
                # update loss file
                losswriter(ii, t_loss, v_loss, lr, epoch)
